In [ ]:
from pathlib import Path
import sys
sys.path.append("../src")
sys.path.append("../src/baseline")


from defs import ROOT, VAL_DATA_PATH, VAL_IMAGES_PATH
from inference.submit import prepare_submit
from inference.grid_search import run_grid_search
from baseline.evaluation.evaluation import calculation_map_050, calculate_meanIOU, competition_metric

import warnings
warnings.filterwarnings("ignore")

In [ ]:
exp_names = ["cascade-R50-2fc-FrozenBN-bs=6", 
             "yolov5/yolov5m6-2048-bs=16", 
            ]
exp_dirs = [ROOT / "outputs" / exp_name for exp_name in exp_names]

In [ ]:
score_threshes = [0.0001] * len(exp_dirs)
nms_threshes = [0.5, 0.6]
tta = False
ensemble_method = "wbf"
ensemble_method_kwargs = {"iou_thr": 0.6}

prepare_submit(VAL_IMAGES_PATH, Path("."), exp_dirs, score_threshes, nms_threshes, tta=tta, ensemble_method=ensemble_method, **ensemble_method_kwargs)
competition_metric(str(VAL_DATA_PATH / "coco.json"), "detection_predictions.json",
                   str(VAL_DATA_PATH / "segmentation_gt.json"), "segmentation_predictions.json")

## Grid search

In [ ]:
exp_names = ["yolov5/yolov5m6-2048-bs=16", "yolov5/yolov5m6-2560-bs=8", "yolov5/yolov5l6-1536-smooth=0.1-bs=16"]
exp_dirs = [ROOT / "outputs" / exp_name for exp_name in exp_names]

In [ ]:
score_threshes = [[0.0001] * len(exp_dirs)]
nms_threshes = [[0.6, 0.5, 0.5]]

ensemble_methods = ["wbf"]
iou_threshes = [0.6]
weights = [[1, 1, 1]]
#ensemble_methods_kwargs = [{"iou_thr": 0.5, "weights": [1, 1, 1, 1]}]
ensemble_methods_kwargs = [{"iou_thr": iou_thr, "weights": w} for iou_thr in iou_threshes for w in weights]
metrics = run_grid_search(VAL_IMAGES_PATH, Path("."), exp_dirs, score_threshes, nms_threshes, ensemble_methods, ensemble_methods_kwargs)